In [2]:
import csv
from csv import DictReader
import random
import math

data_dblp = []
with open('Desktop/Data_D/data.csv', encoding="utf8", errors='ignore') as read_obj:
    dict_reader = DictReader(read_obj)
    data = list(dict_reader)
    for d in data:
        if d['label'] == '0' or d['label'] == '1':
            d['paperAbstract1'] = d['paperAbstract1'].strip()
            d['paperAbstract2'] = d['paperAbstract2'].strip()
            data_dblp.append(d)
        
print(len(data_dblp))
print(data_dblp[0])

random.shuffle(data_dblp)
train_data = data_dblp[0:math.floor(0.75* (len(data_dblp)))]
test_data = data_dblp[math.floor(0.75 * (len(data_dblp))):len(data_dblp)]

print(len(train_data), "***************", len(test_data))


109051
OrderedDict([('id1', '44cc6070-b7e5-4fba-90a0-e0ed84e85775'), ('id2', '01216a98-f91b-4d14-9ae6-59a0ecf0365d'), ('paperAbstract1', 'Regression testing is the verification that previously functioning software remains after a change. In this paper we report on a systematic review of empirical evaluations of regression test selection techniques published in major software engineering journals and conferences. Out of 2923 papers analyzed in this systematic review we identified 28 papers reporting on empirical comparative evaluations of regression test selection techniques. They report on 38 unique studies (23 experiments and 15 case studies) and in total 32 different techniques for regression test selection are evaluated. Our study concludes that no clear picture of the evaluated techniques can be provided based on existing empirical evidence except for a small group of related techniques. Instead we identified a need for more and better empirical studies were concepts are evaluated 

In [3]:
from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model,load_model
import math

In [8]:
pairsData = []

for data in train_data:
    pairsData.append(tuple((data['paperAbstract1'], data['paperAbstract2'], data['label'])))

In [9]:
#print(pairsData)

In [10]:

source_index = {}
#source_index = {data[0]: idx for idx, data in enumerate(pairsData, start = 0)}
m = 0

for idx in range(len(pairsData)):
    if not pairsData[idx][0] in source_index:
        source_index[pairsData[idx][0]] = m
        m = m + 1

print(len(source_index))

'''
temp = []
for k,v in source_index.items():
    temp.append(v)
temp.sort()
print(temp)
'''

45871


'\ntemp = []\nfor k,v in source_index.items():\n    temp.append(v)\ntemp.sort()\nprint(temp)\n'

In [11]:
## Checking
'''
strcheck = []
for idx, data in enumerate(pairsData):
    strcheck.append(data[1])
    
print(len(strcheck))
strcheck = set(strcheck)
print(len(strcheck))
'''

'\nstrcheck = []\nfor idx, data in enumerate(pairsData):\n    strcheck.append(data[1])\n    \nprint(len(strcheck))\nstrcheck = set(strcheck)\nprint(len(strcheck))\n'

In [12]:
#print(len(pairsData))
target_index = {}
#target_index = {data[1]: idx for idx, data in enumerate(pairsData, start = 1)}

m = 0

for idx in range(len(pairsData)):
    if not pairsData[idx][1] in target_index:
        target_index[pairsData[idx][1]] = m
        m = m + 1
 
'''
temp = []
for k,v in target_index.items():
    temp.append(v)
temp.sort()
print(temp)
'''
#index_target = {idx: data for data, idx in target_index.items()}
#print(index_target[1])

'\ntemp = []\nfor k,v in target_index.items():\n    temp.append(v)\ntemp.sort()\nprint(temp)\n'

In [13]:
finalData = []

for data in train_data:
    finalData.append(tuple((source_index[data['paperAbstract1']], target_index[data['paperAbstract2']], data['label'])))
    
#print(finalData)

In [14]:
## For generating negative samples
finalData_Unique = set(finalData)
print(len(finalData_Unique))

78098


In [15]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
embedding_size = 100
def model():
    
    source = Input(name = 'source', shape = [1])
    target = Input(name = 'target', shape = [1])
    
    source_embedding = Embedding(name = 'source_embedding',input_dim = len(source_index),
                               output_dim = embedding_size)(source)
    
    target_embedding = Embedding(name = 'target_embedding',input_dim = len(target_index),
                               output_dim = embedding_size)(target)
    
    final_layer = Reshape([1])(Dot(name = 'dot_product', normalize = True, axes = 2)([source_embedding, target_embedding]))
    
    final_layer = Dense(1, activation = 'sigmoid')(final_layer)
    model = Model(inputs = [source, target], outputs = final_layer)
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy',f1_m,precision_m, recall_m])

    return model

model = model()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
source (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
target (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
source_embedding (Embedding)    (None, 1, 100)       4587100     source[0][0]                     
__________________________________________________________________________________________________
target_embedding (Embedding)    (None, 1, 100)       3748400     target[0][0]                     
_______________________________________________________________________________________

In [17]:
import numpy as np
import random
random.seed(1000)

def batchifier(finalData, batch_size, positive_samples, negative_samples):
    batch = np.zeros((batch_size, 3))    
    while True:

        for idx, (source_id, target_id, label) in enumerate(random.sample(finalData, batch_size)):
            #if label != ' ':
            batch[idx, :] = (source_id, target_id, label)

        '''
        idx = idx + 1
        while idx < batch_size:
            
            src = random.randrange(len(source_index))
            tar = random.randrange(len(target_index))
            if (src, tar) not in finalData_Unique:
                batch[idx, :] = (src, tar, 0)
                idx = idx + 1     
        '''
        np.random.shuffle(batch)
        yield {'source': batch[:, 0], 'target': batch[:, 1]}, batch[:, 2]

In [18]:
batch_size = 1024
positive_samples = math.ceil(batch_size * 0.7)
negative_samples = math.floor(batch_size * 0.3)

print(positive_samples)
print(negative_samples)
gen = batchifier(finalData, batch_size, positive_samples, negative_samples)

717
307


In [19]:
# Generating Batch
next(batchifier(finalData, batch_size, positive_samples = 2, negative_samples = 2))

({'source': array([22976.,  1647., 17139., ...,   701., 24736., 28875.]),
  'target': array([12915.,   199., 15328., ...,   553., 21470., 24745.])},
 array([1., 0., 1., ..., 0., 1., 1.]))

In [21]:
# Train
# Steps_per_epoch = Training Size - too long
h = model.fit(gen, epochs = 2,steps_per_epoch = 10, verbose = 2)

Epoch 1/2
10/10 - 1s - loss: 0.6730 - accuracy: 0.6185 - f1_m: 0.6925 - precision_m: 0.6761 - recall_m: 0.7102
Epoch 2/2
10/10 - 1s - loss: 0.6633 - accuracy: 0.6471 - f1_m: 0.7182 - precision_m: 0.6840 - recall_m: 0.7561


In [22]:
model.save('Desktop/word2vecModel_Dblp')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Desktop/word2vecModel_Dblp/assets


In [26]:
## Test
import pandas as pd
pairsDataTest = []
batch_size = 1024

test_data = test_data[:1024]
for data in test_data:
    pairsDataTest.append(tuple((data['paperAbstract1'], data['paperAbstract2'], data['label'])))

source_index_test = {}
m = 0

for idx in range(len(pairsDataTest)):
    if not pairsDataTest[idx][0] in source_index_test:
        source_index_test[pairsDataTest[idx][0]] = m
        m = m + 1

target_index_test = {}
m = 0

for idx in range(len(pairsDataTest)):
    if not pairsDataTest[idx][1] in target_index_test:
        target_index_test[pairsDataTest[idx][1]] = m
        m = m + 1

finalDataTest = []

for data in test_data:
    finalDataTest.append(tuple((source_index_test[data['paperAbstract1']], target_index_test[data['paperAbstract2']], data['label'])))
    
#print(finalDataTest)
gen_test = batchifier(finalDataTest, batch_size, positive_samples = 2, negative_samples = 2)
x,y = next(batchifier(finalData, batch_size, positive_samples = 2, negative_samples = 2))

# Keeping Default Batch Size
predictions = model.predict(gen_test, verbose = 2, steps = batch_size)
#print(predictions)
y_classes = predictions.argmax(axis=-1)
df = pd.DataFrame(y_classes)
print("=======Predictions=======")
print(df)

## Retriving the Score
score = model.evaluate(gen_test,verbose = 1, steps = batch_size)
print(score)

1024/1024 - 7s
=======Predictions=======
         0
0        0
1        0
2        0
3        0
4        0
...     ..
1048571  0
1048572  0
1048573  0
1048574  0
1048575  0

[1048576 rows x 1 columns]
1024/1024 [==============================] - 8s 7ms/step - loss: 0.6898 - accuracy: 0.5400 - f1_m: 0.6402 - precision_m: 0.6099 - recall_m: 0.6736
[0.6898403763771057, 0.5400390625, 0.6401917934417725, 0.6099004745483398, 0.673639714717865]


In [32]:
## Visualizations
from sklearn.manifold import TSNE